# Project Day 12: RFM Analysis - Feature Engineering

**Objective:** Create an RFM (Recency, Frequency, Monetary) customer segmentation model.

The challenge is to transform a transaction log (where each row is an *item*) into a customer 
table (where each row is a *customer*), by creating the 3 main features:
1.  **Recency (R):** How many days ago was their last purchase?
2.  **Frequency (F):** How many (unique invoice) purchases have they made?
3.  **Monetary (M):** How much have they spent in total?

In [39]:
import pandas as pd
import numpy as np
import datetime as dt # I'll use this to create the Timedelta

## 1. Data Load and Cleaning

I will load the dataset, take an initial look, and perform the necessary cleaning.

In [40]:
# I had to use encoding='latin1' (or 'ISO-8859-1') because the default 'utf-8'
# doesn't recognize some special characters in the file, like '£'.

df = pd.read_csv('Online_Retail.csv', encoding='latin1')
print("Dataset loaded successfully!")

Dataset loaded successfully!


In [41]:
# Taking a quick look at the structure
print("\n--- Initial DataFrame Info ---")
print(df.info())
print("\n--- First 5 Rows ---")
print(df.head())


--- Initial DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None

--- First 5 Rows ---
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATE

In [42]:
# --- Cleaning ---
# I'll operate on a copy to keep the original safe
df_cleaned = df.copy()

# 1. Remove canceled transactions (Quantity < 0)
df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]

# 2. Remove anonymous customers (CustomerID is null)
# I can't perform customer segmentation without a customer ID.
df_cleaned = df_cleaned.dropna(subset=['CustomerID'])

print(f"\nData cleaned. Rows before: {len(df)}, Rows now: {len(df_cleaned)}")


Data cleaned. Rows before: 541909, Rows now: 397924


## 2. Feature Engineering (Row-Level)

Here, I'll create new features using information from the *same row*.

In [46]:
# 1. Create the 'TotalValue' column
# This is the first feature I'm engineering.
df_cleaned['TotalValue'] = df_cleaned['Quantity'] * df_cleaned['UnitPrice']

# 2. Convert 'InvoiceDate' to datetime format
# Pandas read it as text (object), I need to convert it to a date
# to be able to perform time calculations (Recency).
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'], dayfirst=True)

print("\n--- DataFrame after Row-Level Engineering ---")
display(df_cleaned.head())


--- DataFrame after Row-Level Engineering ---


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalValue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,20.34


## 3. Feature Engineering (Aggregation-Level - RFM)

This is the main part. I will "compress" each customer's history into 3 metrics, using `.groupby()`.
"""

In [44]:
# Defining our dataset's "today".
# I'll calculate the date of the last purchase in the dataset and add 1 day.
# This ensures that no one's recency will be zero.
snapshot_date = df_cleaned['InvoiceDate'].max() + pd.Timedelta(days=1)

print(f"\nSnapshot Date ('Today'): {snapshot_date.date()}")

# 4.2. The RFM magic: .groupby() with .agg()
# I'll group by customer and calculate all 3 features at once.
rfm_df = df_cleaned.groupby('CustomerID').agg(
    
    # Recency: I get the max date (last purchase) for each customer
    # and subtract it from the 'snapshot_date' I created.
    Recency=('InvoiceDate', lambda x: (snapshot_date - x.max()).days),
    
    # Frequency: I count how many unique 'InvoiceNo' each customer has.
    Frequency=('InvoiceNo', 'nunique'),
    
    # Monetary: I simply sum all the 'TotalValue' for that customer.
    Monetary=('TotalValue', 'sum')
)

# Adjusting CustomerID to be a column and of type 'int'
rfm_df = rfm_df.reset_index().astype({'CustomerID': int})


Snapshot Date ('Today'): 2011-12-11


## 4. Final Result

Now we have a clean, aggregated DataFrame, where each row represents a single customer 
and their behavioral metrics.

In [45]:
print("\n--- Final RFM Table (Top 5 Customers) ---")
print(rfm_df.head())


--- Final RFM Table (Top 5 Customers) ---
   CustomerID  Recency  Frequency  Monetary
0       12346      327          1  77183.60
1       12347       41          7   4310.00
2       12348       77          4   1797.24
3       12349       20          1   1757.55
4       12350      312          1    334.40
